In [ ]:
# # Required Packages for the Project RUN ONLY ONCE
# %pip install matplotlib
# %pip install nilearn
# %pip install openpyxl
# %pip install Path
# %pip install seaborn
# %pip install nltools
# %pip install scipy
# %pip install scikit-image
# %pip install nibabel

In [87]:
# import copy
# from skimage import filters
# import json
# import nibabel as nib
# from nilearn.plotting import plot_img

import pandas as pd
from nilearn import image
from nilearn.image import load_img, index_img
import numpy as np
import scipy.io
import networkx as nx
import matplotlib as mt
import scipy as sc

In [96]:
class SubjectData:
    
    def __init__(self, datasetPath, componentsPath):
        self.componentFileName = "adni_aa__sub01_component_ica_s1_.nii"
        self.timeCourseFileName = "adni_aa__sub01_timecourses_ica_s1_.nii"
        self.FNCmatFile = "adni_aa__postprocess_results.mat"
        self.component_key = "fnc_corrs_all"
        self.graphMetricNames = [
                "Global efficiency", 
                "Characteristic path length", 
                "Clustering coefficient",
                # ""
            ]
        
        self.subjectsData = self.readFileofCSV(datasetPath) 
        self.componentData = self.readFileofCSV(componentsPath)

        self.modifySubjectsPath()

        self.domainList = self.componentData["icn_domain"]
        self.indexList = self.componentData["icn_index"]

    def modifySubjectsPath(self):
        for i in range(2404):
            self.subjectsData.at[i,"fc_dir"] = self.subjectsData.iloc[i]["fc_dir"].replace("FC","GIGICA")

    def readFileofCSV(self,path):
        fileData = pd.read_csv(path)
        return fileData
    
    def getDatasetPaths(self,subjectName):
        ans = list()
        for i in range(2404):
            if self.subjectsData.at[i,"ResearchGroup"] == subjectName:
                ans.append(self.subjectsData.at[i,"fc_dir"])
        return ans

In [97]:
class VoxelCounts(SubjectData):
    " \
        datasetPath : 4D images path \
        componentsPath: indexes path \
        subjectList: List of subjects to calculate ex: AD, CN, ....  \
    "
    def __init__(self,datasetPath, componentsPath, subjectList):
        super().__init__(datasetPath,componentsPath)
        self.subjectsToCalculate = subjectList

        self.voxelCountMap = dict()
        self.prepareVoxelCountMap()

    def prepareVoxelCountMap(self):
        " \
            iterate over the list of Subjects like AD, CN, ....     \
        "
        for subjectName in self.subjectsToCalculate:
            self.voxelCountMap[subjectName]=dict()
            self.voxelCountMap[subjectName]["paths"] = self.getDatasetPaths(subjectName)
            self.voxelCountMap[subjectName]["indexes"] = dict()
            for i in self.indexList:
                self.voxelCountMap[subjectName]["indexes"][i]=list()

    def calculateVoxelCount(self):
        for subjectName in self.subjectsToCalculate:
            for path in self.voxelCountMap[subjectName]["paths"]:
                spacialMapName = path + self.componentFileName
                spacialMap = load_img(spacialMapName)
                for index in self.voxelCountMap[subjectName]["indexes"]:
                    actualIndex=index-1
                    componentImg = index_img(spacialMap, actualIndex)
                    componentImgData = componentImg.get_fdata()
                    component_threshold = 3*np.std(componentImgData)
                    component_voxelCount = np.count_nonzero(componentImgData > component_threshold)
                    self.voxelCountMap[subjectName]["indexes"][index].append(component_voxelCount)

In [98]:
AD_Threshold = VoxelCounts('ADNI_demos.txt', 'NM_icns_info.csv', ["AD","CN"])
AD_Threshold.calculateVoxelCount()

In [99]:
class GraphMetrics(SubjectData):

    " \
        datasetPath : 4D images path \
        componentsPath: indexes path \
        subjectList: List of subjects to calculate ex: AD, CN, ....  \
    "
    
    def __init__(self,datasetPath, componentsPath, subjectList):
        super().__init__(datasetPath,componentsPath)
        self.subjectsToCalculate = subjectList

        self.graphMetricMap = dict()
        self.prepareGraphMetricMap()

    def prepareGraphMetricMap(self):
        for subjectName in self.subjectsToCalculate:
            self.graphMetricMap[subjectName]=dict()
            self.graphMetricMap[subjectName]["paths"] = self.getDatasetPaths(subjectName)
            for graphMetic in self.graphMetricNames:
                self.graphMetricMap[subjectName][graphMetic]=list()

    def loadMatFile(self,filePath):
        componentDict = scipy.io.loadmat(filePath+self.FNCmatFile)
        return componentDict[self.component_key]

    def prepareFNCMatrix(self,componentData):
        selected_component = np.zeros((53,53), dtype=np.float64).reshape(53,53)
        for i in range(53):
            for j in range(i+1, 53):
                # if componentData[self.indexList[i]-1][self.indexList[j]-1] >=0 : 
                selected_component[i][j]=componentData[self.indexList[i]-1][self.indexList[j]-1]
        selected_component += selected_component.T

        # Finding correlation matrix
        corrs = pd.DataFrame(selected_component)
        correlation_matrix = corrs.corr()
        correlation_numpy_array = correlation_matrix.to_numpy(dtype=np.float64)

        # Finding Thresholded correlation matrix
        row,column = correlation_numpy_array.shape
        for i in range(row):
            for j in range(column):
                if i!=j and correlation_numpy_array[i][j]<0:
                    correlation_numpy_array[i][j]=0

        input_matrix = nx.from_numpy_array(correlation_numpy_array)
        return input_matrix

    def calculateGraphMetrics(self):
        for subjectName in self.subjectsToCalculate:
            for path in self.graphMetricMap[subjectName]["paths"]:
                fncMatrix = self.prepareFNCMatrix(self.loadMatFile(path))
                for graphMetric in self.graphMetricNames:
                    if graphMetric == "Global efficiency":
                        globalEfficieny = nx.global_efficiency(fncMatrix)
                        self.graphMetricMap[subjectName][graphMetric].append(globalEfficieny)

                    elif graphMetric == "Characteristic path length":
                        characteristicPathLength = nx.average_shortest_path_length(fncMatrix, weight='weight')
                        self.graphMetricMap[subjectName][graphMetric].append(characteristicPathLength)

                    elif graphMetric == "Clustering coefficient":
                        clusterCofficient = nx.clustering(fncMatrix, weight='weight')
                        clusterCofficient_network = 0
                        for i in clusterCofficient:
                            clusterCofficient_network += clusterCofficient[i]
                        clusterCofficient_network /= 53
                        self.graphMetricMap[subjectName][graphMetric].append(clusterCofficient_network)

In [100]:
AD_GraphMetric = GraphMetrics('ADNI_demos.txt', 'NM_icns_info.csv', ["AD","CN"])
AD_GraphMetric.calculateGraphMetrics()

In [110]:
# table.columns = column_names
# table.index = AD_GraphMetric.graphMetricNames

# table

# correlation = table["Characteristic path length"].corr(table[69])
# sns.regplot(x=table["Global efficiency"], y=table[AD_Threshold.domainList[0]])
# sns.regplot(x=table["Global efficiency"], y=table[AD_Threshold.domainList[6]])

GraphMetrics_Names = AD_GraphMetric.graphMetricNames
ans1 = list()

for subjectName in AD_GraphMetric.subjectsToCalculate:

    table = pd.DataFrame()
    for i in range(len(AD_Threshold.indexList)):
        table[AD_Threshold.domainList[i]+'('+str(AD_Threshold.indexList[i])+')'] = list()

    for i in GraphMetrics_Names:
        indexList = AD_Threshold.indexList
        corr_list = list()

        xlist = AD_GraphMetric.graphMetricMap[subjectName][i]
        xmean = np.mean(xlist)
        xval = xlist - xmean
        xsqu = np.sqrt(np.sum(np.square(xval)))

        for j in range(len(indexList)):

            ylist = AD_Threshold.voxelCountMap[subjectName]["indexes"][AD_Threshold.indexList[j]]
            ymean = np.mean(ylist)
            yval = ylist - ymean
            ysqu = np.sqrt(np.sum(np.square(yval)))

            num = 0
            for k in range(len(yval)):
                num += (xval[k]*yval[k])

            den = (xsqu * ysqu)
            ans = num/den
            corr_list.append(ans)

        table.loc[len(table.index)] = corr_list
    table.index = AD_GraphMetric.graphMetricNames
    ans1.append(table)

In [146]:
temp = ans1

temp[0].to_csv('./Alzhiemeris.csv')
temp[1].to_csv('./Controls.csv')

In [147]:
# ans1[1].to_csv('./Controls.csv')

columns = ans1[1].columns.values

rows = ans1[1].index.values

# print(ans1[0][rows[0]][columns[0]] - ans1[1][rows[0]][columns[0]])

for i in rows:
    for j in columns:
        ans1[1].at[i,j] -= ans1[0].at[i,j]

ans1[1].to_csv('./difference.csv')